In [19]:
import os 
import cv2
from tqdm import tqdm
import xml.etree.ElementTree as ET
import shutil

Copy and resize original PDS jpg files

In [15]:
ORI_IMG_DIR = r"F:\Minor Data Collection\Image Data Collection\Patan Durbar Square\PDS Original Renamed Collected Images"
DEST_IMG_DIR = r"F:\Minor Data Collection\Final Image Data\Patan Durbar Square\PDS Data Annotate 512\JPEGImages"
COPY_IMG_DIR = r"F:\Minor Data Collection\Final Image Data\Patan Durbar Square\PDS Data Annotate 300\JPEGImages"

ANNO_INPUT_DIR = r"F:\Minor Data Collection\Final Image Data\Patan Durbar Square\PDS Data Annotate 300\Annotations"
ANNO_OUTPUT_DIR = r"F:\Minor Data Collection\Final Image Data\Patan Durbar Square\PDS Data Annotate 512\Annotations"

In [22]:
temp_list = list()
output_dim = (512, 512)

outer_loop_count = 0
for img_file in tqdm(os.listdir(COPY_IMG_DIR)):
    if os.path.exists(os.path.join(ORI_IMG_DIR, img_file)):
        image_file_name = os.path.join(ORI_IMG_DIR, img_file)
        img = cv2.imread(image_file_name)
        img_height, img_width, _ = img.shape
        
        resized_img = cv2.resize(img, output_dim, fx = 0.5, fy = 0.5,  interpolation = cv2.INTER_AREA)
        cv2.imwrite(os.path.join(DEST_IMG_DIR, img_file), resized_img)
        
        xml_input_file = os.path.join(ANNO_INPUT_DIR, img_file.split('.')[0] + '.xml')
        xml_file = ET.parse(xml_input_file)
        root = xml_file.getroot()
        
        root.find('size').find('width').text = str(output_dim[0])
        root.find('size').find('height').text = str(output_dim[0])
        
        all_objects = root.findall('object')
        
        for member in all_objects:
            xmin= int(member[4][0].text)
            ymin= int(member[4][1].text)
            xmax= int(member[4][2].text)
            ymax= int(member[4][3].text)

            member[4][0].text = str(int((float(xmin) / 300.0) * 512.0))
            member[4][1].text = str(int((float(ymin) / 300.0) * 512.0))
            member[4][2].text = str(int((float(xmax) / 300.0) * 512.0))
            member[4][3].text = str(int((float(ymax) / 300.0) * 512.0))
        
        anno_output_path = os.path.join(ANNO_OUTPUT_DIR, img_file.split('.')[0] + '.xml')
        xml_file.write(anno_output_path)
    outer_loop_count += 1

100%|██████████████████████████████████████████████████████████████████████████████| 2977/2977 [10:30<00:00,  4.72it/s]


2977